In [9]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("phuocsang/hoidap-tvpl-20k")

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'url', 'keyword', 'time_published', 'date_published', 'type', 'author', '__index_level_0__'],
        num_rows: 19965
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'url', 'keyword', 'time_published', 'date_published', 'type', 'author', '__index_level_0__'],
        num_rows: 2035
    })
})

In [11]:
# Phân tích cấu trúc dữ liệu chi tiết
import pandas as pd

print("📊 Thông tin dataset:")
print(f"- Train set: {len(ds['train'])} samples")
print(f"- Test set: {len(ds['test'])} samples")
print(f"- Total: {len(ds['train']) + len(ds['test'])} samples")

print("\n📝 Các trường có sẵn:")
for feature in ds['train'].features:
    print(f"- {feature}")

print("\n🔍 Xem mẫu dữ liệu đầu tiên:")
sample = ds['train'][0]
for key, value in sample.items():
    if isinstance(value, str) and len(value) > 100:
        print(f"{key}: {value[:100]}...")
    else:
        print(f"{key}: {value}")

📊 Thông tin dataset:
- Train set: 19965 samples
- Test set: 2035 samples
- Total: 22000 samples

📝 Các trường có sẵn:
- id
- question
- answer
- url
- keyword
- time_published
- date_published
- type
- author
- __index_level_0__

🔍 Xem mẫu dữ liệu đầu tiên:
id: 35a9fd5e-b75e-42ff-a268-bb0a895251bf
question: Trong Bộ luật Hình sự thì bao nhiêu tuổi được xem là người già, người cao tuổi?
answer: Người cao tuổi, người già, người già yếu được quy định tại Điều 2 Luật Người cao tuổi 2009: "Người c...
url: https://thuvienphapluat.vn/phap-luat/nguoi-71-tuoi-pham-toi-phai-chiu-trach-nhiem-hinh-su-thi-co-duo...
keyword: ['Người cao tuổi', 'Trách nhiệm hình sự']
time_published: 06/05/2022
date_published: 14:02
type: thu-tuc-to-tung
author: Nguyễn Khánh Huyền
__index_level_0__: 349345


In [12]:
# Phân tích chất lượng dữ liệu question và answer
import numpy as np

def analyze_text_quality(dataset_split, split_name):
    """Phân tích chất lượng text trong dataset"""
    questions = [item['question'] for item in dataset_split]
    answers = [item['answer'] for item in dataset_split]
    
    print(f"\n📈 Phân tích {split_name}:")
    
    # Độ dài câu hỏi
    question_lengths = [len(q) if q else 0 for q in questions]
    print(f"🔸 Độ dài câu hỏi:")
    print(f"   - Trung bình: {np.mean(question_lengths):.1f} ký tự")
    print(f"   - Min: {min(question_lengths)}, Max: {max(question_lengths)}")
    print(f"   - Median: {np.median(question_lengths):.1f}")
    
    # Độ dài câu trả lời
    answer_lengths = [len(a) if a else 0 for a in answers]
    print(f"🔸 Độ dài câu trả lời:")
    print(f"   - Trung bình: {np.mean(answer_lengths):.1f} ký tự")
    print(f"   - Min: {min(answer_lengths)}, Max: {max(answer_lengths)}")
    print(f"   - Median: {np.median(answer_lengths):.1f}")
    
    # Kiểm tra dữ liệu rỗng
    empty_questions = sum(1 for q in questions if not q or q.strip() == "")
    empty_answers = sum(1 for a in answers if not a or a.strip() == "")
    
    print(f"🔸 Dữ liệu rỗng:")
    print(f"   - Câu hỏi rỗng: {empty_questions}")
    print(f"   - Câu trả lời rỗng: {empty_answers}")
    
    return {
        'questions': questions,
        'answers': answers,
        'question_lengths': question_lengths,
        'answer_lengths': answer_lengths,
        'empty_questions': empty_questions,
        'empty_answers': empty_answers
    }

# Phân tích train set
train_analysis = analyze_text_quality(ds['train'], "Train Set")

# Phân tích test set  
test_analysis = analyze_text_quality(ds['test'], "Test Set")


📈 Phân tích Train Set:
🔸 Độ dài câu hỏi:
   - Trung bình: 88.7 ký tự
   - Min: 2, Max: 373
   - Median: 87.0
🔸 Độ dài câu trả lời:
   - Trung bình: 1792.2 ký tự
   - Min: 55, Max: 765892
   - Median: 1497.0
🔸 Dữ liệu rỗng:
   - Câu hỏi rỗng: 0
   - Câu trả lời rỗng: 0

📈 Phân tích Test Set:
🔸 Độ dài câu hỏi:
   - Trung bình: 89.7 ký tự
   - Min: 9, Max: 218
   - Median: 88.0
🔸 Độ dài câu trả lời:
   - Trung bình: 1758.9 ký tự
   - Min: 132, Max: 238002
   - Median: 1499.0
🔸 Dữ liệu rỗng:
   - Câu hỏi rỗng: 0
   - Câu trả lời rỗng: 0


In [13]:
# Làm sạch và chuẩn hóa dữ liệu
def clean_text(text):
    """Làm sạch text"""
    if not text:
        return ""
    
    # Loại bỏ khoảng trắng thừa
    text = " ".join(text.split())
    
    # Loại bỏ ký tự đặc biệt không cần thiết
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    
    return text.strip()

def process_dataset(dataset_split, max_answer_length=5000):
    """Xử lý dataset và lọc dữ liệu chất lượng"""
    processed_data = []
    
    for item in dataset_split:
        question = clean_text(item['question'])
        answer = clean_text(item['answer'])
        
        # Lọc dữ liệu theo tiêu chí chất lượng
        if (len(question) >= 10 and  # Câu hỏi ít nhất 10 ký tự
            len(answer) >= 50 and    # Câu trả lời ít nhất 50 ký tự
            len(answer) <= max_answer_length and  # Giới hạn độ dài trả lời
            question.endswith('?')):  # Câu hỏi phải kết thúc bằng dấu ?
            
            processed_data.append({
                'question': question,
                'answer': answer
            })
    
    return processed_data

print("🔧 Đang xử lý và làm sạch dữ liệu...")

# Xử lý train set
train_processed = process_dataset(ds['train'])
print(f"✅ Train set: {len(ds['train'])} → {len(train_processed)} (giữ lại {len(train_processed)/len(ds['train'])*100:.1f}%)")

# Xử lý test set
test_processed = process_dataset(ds['test'])
print(f"✅ Test set: {len(ds['test'])} → {len(test_processed)} (giữ lại {len(test_processed)/len(ds['test'])*100:.1f}%)")

print(f"\n📊 Tổng kết sau xử lý:")
print(f"- Tổng số mẫu: {len(train_processed) + len(test_processed)}")
print(f"- Train: {len(train_processed)} mẫu")
print(f"- Test: {len(test_processed)} mẫu")

🔧 Đang xử lý và làm sạch dữ liệu...
✅ Train set: 19965 → 19536 (giữ lại 97.9%)
✅ Test set: 2035 → 1993 (giữ lại 97.9%)

📊 Tổng kết sau xử lý:
- Tổng số mẫu: 21529
- Train: 19536 mẫu
- Test: 1993 mẫu
✅ Train set: 19965 → 19536 (giữ lại 97.9%)
✅ Test set: 2035 → 1993 (giữ lại 97.9%)

📊 Tổng kết sau xử lý:
- Tổng số mẫu: 21529
- Train: 19536 mẫu
- Test: 1993 mẫu


In [14]:
# Tạo thư mục và lưu dữ liệu
import os
import json

# Tạo thư mục data/finetune_data
output_dir = "../data/finetune_data"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Tạo thư mục: {output_dir}")

# Lưu dữ liệu dưới nhiều format khác nhau

# 1. Format JSONL (mỗi dòng là một JSON object)
def save_jsonl(data, filepath):
    """Lưu dữ liệu dưới định dạng JSONL"""
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# 2. Format instruction (cho fine-tuning)
def save_instruction_format(data, filepath):
    """Lưu dữ liệu dưới định dạng instruction tuning"""
    instruction_data = []
    for item in data:
        instruction_item = {
            "instruction": "Trả lời câu hỏi pháp luật sau:",
            "input": item['question'],
            "output": item['answer']
        }
        instruction_data.append(instruction_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in instruction_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# 3. Format conversation (cho chatbot training)
def save_conversation_format(data, filepath):
    """Lưu dữ liệu dưới định dạng conversation"""
    conversation_data = []
    for item in data:
        conversation_item = {
            "conversations": [
                {"role": "user", "content": item['question']},
                {"role": "assistant", "content": item['answer']}
            ]
        }
        conversation_data.append(conversation_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in conversation_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

print("💾 Đang lưu dữ liệu...")

# Lưu train set
train_files = {
    'qa_format.jsonl': train_processed,
    'instruction_format.jsonl': None,  # Will be handled by save_instruction_format
    'conversation_format.jsonl': None  # Will be handled by save_conversation_format
}

# Lưu QA format
train_qa_path = os.path.join(output_dir, "train_qa_format.jsonl")
save_jsonl(train_processed, train_qa_path)
print(f"✅ Đã lưu train QA format: {train_qa_path}")

# Lưu instruction format
train_instruction_path = os.path.join(output_dir, "train_instruction_format.jsonl")
save_instruction_format(train_processed, train_instruction_path)
print(f"✅ Đã lưu train instruction format: {train_instruction_path}")

# Lưu conversation format
train_conversation_path = os.path.join(output_dir, "train_conversation_format.jsonl")
save_conversation_format(train_processed, train_conversation_path)
print(f"✅ Đã lưu train conversation format: {train_conversation_path}")

# Lưu test set
test_qa_path = os.path.join(output_dir, "test_qa_format.jsonl")
save_jsonl(test_processed, test_qa_path)
print(f"✅ Đã lưu test QA format: {test_qa_path}")

test_instruction_path = os.path.join(output_dir, "test_instruction_format.jsonl")
save_instruction_format(test_processed, test_instruction_path)
print(f"✅ Đã lưu test instruction format: {test_instruction_path}")

test_conversation_path = os.path.join(output_dir, "test_conversation_format.jsonl")
save_conversation_format(test_processed, test_conversation_path)
print(f"✅ Đã lưu test conversation format: {test_conversation_path}")

📁 Tạo thư mục: ../data/finetune_data
💾 Đang lưu dữ liệu...
✅ Đã lưu train QA format: ../data/finetune_data\train_qa_format.jsonl
✅ Đã lưu train QA format: ../data/finetune_data\train_qa_format.jsonl
✅ Đã lưu train instruction format: ../data/finetune_data\train_instruction_format.jsonl
✅ Đã lưu train instruction format: ../data/finetune_data\train_instruction_format.jsonl
✅ Đã lưu train conversation format: ../data/finetune_data\train_conversation_format.jsonl
✅ Đã lưu test QA format: ../data/finetune_data\test_qa_format.jsonl
✅ Đã lưu test instruction format: ../data/finetune_data\test_instruction_format.jsonl
✅ Đã lưu test conversation format: ../data/finetune_data\test_conversation_format.jsonl
✅ Đã lưu train conversation format: ../data/finetune_data\train_conversation_format.jsonl
✅ Đã lưu test QA format: ../data/finetune_data\test_qa_format.jsonl
✅ Đã lưu test instruction format: ../data/finetune_data\test_instruction_format.jsonl
✅ Đã lưu test conversation format: ../data/finetu

In [15]:
# Tạo metadata và tổng kết
metadata = {
    "dataset_info": {
        "source": "phuocsang/hoidap-tvpl-20k",
        "description": "Vietnamese Legal Q&A Dataset processed for fine-tuning",
        "total_samples": len(train_processed) + len(test_processed),
        "train_samples": len(train_processed),
        "test_samples": len(test_processed)
    },
    "processing_info": {
        "filters_applied": [
            "Minimum question length: 10 characters",
            "Minimum answer length: 50 characters", 
            "Maximum answer length: 5000 characters",
            "Questions must end with '?'",
            "Text cleaning: removed extra whitespace and special characters"
        ],
        "retention_rate": f"{(len(train_processed) + len(test_processed))/(len(ds['train']) + len(ds['test']))*100:.1f}%"
    },
    "file_formats": {
        "qa_format": "Simple question-answer pairs",
        "instruction_format": "Instruction tuning format with instruction/input/output",
        "conversation_format": "Multi-turn conversation format for chatbot training"
    },
    "sample_counts": {
        "train": len(train_processed),
        "test": len(test_processed)
    }
}

# Lưu metadata
metadata_path = os.path.join(output_dir, "metadata.json")
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Đã lưu metadata: {metadata_path}")

# Hiển thị mẫu dữ liệu từ mỗi format
print("\n📝 Mẫu dữ liệu các format:")

print("\n🔸 QA Format:")
sample_qa = train_processed[0]
print(f"Question: {sample_qa['question'][:100]}...")
print(f"Answer: {sample_qa['answer'][:100]}...")

print("\n🔸 Instruction Format:")
print(f"Instruction: Trả lời câu hỏi pháp luật sau:")
print(f"Input: {sample_qa['question'][:100]}...")
print(f"Output: {sample_qa['answer'][:100]}...")

print("\n🔸 Conversation Format:")
print(f"User: {sample_qa['question'][:100]}...")
print(f"Assistant: {sample_qa['answer'][:100]}...")

print(f"\n🎉 Hoàn thành! Đã xử lý và lưu {len(train_processed) + len(test_processed)} mẫu dữ liệu vào {output_dir}")

# Hiển thị thống kê file
print(f"\n📁 Files đã tạo:")
for filename in os.listdir(output_dir):
    filepath = os.path.join(output_dir, filename)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"   - {filename}: {size_mb:.2f} MB")

✅ Đã lưu metadata: ../data/finetune_data\metadata.json

📝 Mẫu dữ liệu các format:

🔸 QA Format:
Question: Trong Bộ luật Hình sự thì bao nhiêu tuổi được xem là người già, người cao tuổi?...
Answer: Người cao tuổi, người già, người già yếu được quy định tại Điều 2 Luật Người cao tuổi 2009: "Người c...

🔸 Instruction Format:
Instruction: Trả lời câu hỏi pháp luật sau:
Input: Trong Bộ luật Hình sự thì bao nhiêu tuổi được xem là người già, người cao tuổi?...
Output: Người cao tuổi, người già, người già yếu được quy định tại Điều 2 Luật Người cao tuổi 2009: "Người c...

🔸 Conversation Format:
User: Trong Bộ luật Hình sự thì bao nhiêu tuổi được xem là người già, người cao tuổi?...
Assistant: Người cao tuổi, người già, người già yếu được quy định tại Điều 2 Luật Người cao tuổi 2009: "Người c...

🎉 Hoàn thành! Đã xử lý và lưu 21529 mẫu dữ liệu vào ../data/finetune_data

📁 Files đã tạo:
   - metadata.json: 0.00 MB
   - test_conversation_format.jsonl: 4.39 MB
   - test_instruction_format.jsonl: 4

In [16]:
# Kiểm tra và validate dữ liệu đã lưu
def validate_jsonl_file(filepath, expected_count):
    """Kiểm tra tính toàn vẹn của file JSONL"""
    try:
        count = 0
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                json.loads(line.strip())  # Kiểm tra JSON hợp lệ
                count += 1
        
        if count == expected_count:
            print(f"✅ {os.path.basename(filepath)}: {count} dòng (OK)")
            return True
        else:
            print(f"❌ {os.path.basename(filepath)}: {count} dòng (Expected: {expected_count})")
            return False
    except Exception as e:
        print(f"❌ Error validating {os.path.basename(filepath)}: {e}")
        return False

print("🔍 Kiểm tra tính toàn vẹn dữ liệu:")

# Validate train files
train_files = [
    "train_qa_format.jsonl",
    "train_instruction_format.jsonl", 
    "train_conversation_format.jsonl"
]

test_files = [
    "test_qa_format.jsonl",
    "test_instruction_format.jsonl",
    "test_conversation_format.jsonl"
]

all_valid = True

print("\n📂 Train files:")
for filename in train_files:
    filepath = os.path.join(output_dir, filename)
    if not validate_jsonl_file(filepath, len(train_processed)):
        all_valid = False

print("\n📂 Test files:")
for filename in test_files:
    filepath = os.path.join(output_dir, filename)
    if not validate_jsonl_file(filepath, len(test_processed)):
        all_valid = False

# Kiểm tra metadata
metadata_file = os.path.join(output_dir, "metadata.json")
try:
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata_loaded = json.load(f)
    print(f"✅ metadata.json: OK")
except Exception as e:
    print(f"❌ metadata.json: Error - {e}")
    all_valid = False

if all_valid:
    print(f"\n🎉 Tất cả files đều hợp lệ! Dataset đã sẵn sàng để fine-tuning.")
    print(f"\n💡 Sử dụng:")
    print(f"   - QA format: Cho traditional Q&A training")
    print(f"   - Instruction format: Cho instruction-following models") 
    print(f"   - Conversation format: Cho chatbot training")
else:
    print(f"\n⚠️ Có lỗi xảy ra. Vui lòng kiểm tra lại.")

print(f"\n📍 Đường dẫn dữ liệu: {os.path.abspath(output_dir)}")

🔍 Kiểm tra tính toàn vẹn dữ liệu:

📂 Train files:
✅ train_qa_format.jsonl: 19536 dòng (OK)
✅ train_instruction_format.jsonl: 19536 dòng (OK)
✅ train_conversation_format.jsonl: 19536 dòng (OK)

📂 Test files:
✅ test_qa_format.jsonl: 1993 dòng (OK)
✅ test_instruction_format.jsonl: 1993 dòng (OK)
✅ test_conversation_format.jsonl: 1993 dòng (OK)
✅ metadata.json: OK

🎉 Tất cả files đều hợp lệ! Dataset đã sẵn sàng để fine-tuning.

💡 Sử dụng:
   - QA format: Cho traditional Q&A training
   - Instruction format: Cho instruction-following models
   - Conversation format: Cho chatbot training

📍 Đường dẫn dữ liệu: d:\project\project_1\Vietnamese-Legal-Chatbot-RAG-System\data_pipeline\data\finetune_data
